# HANA Cloud - Predicitive Analysis Library Hands On


### Documentation
- SAP HANA Python Client API for Machine Learning Algorithms: https://pypi.org/project/hana-ml/

- SAP HANA Predictive Analysis Library (PAL): https://help.sap.com/viewer/2cfbc5cf2bc14f028cfbe2a2bba60a50/1.0.12/en-US

SAP HANA ML Library
You will be using the 'SAP HANA Python Client API for Machine Learning Algorithm'.

### 1.	How can we directly access the data in our HANA? 

In [ ]:
!pip install hana_ml

In [ ]:
import hana_ml

In [ ]:
hana_address = #your hostname as string
hana_port = #your port as integer
hana_user = #your user as string
hana_password = #your password as string
hana_encrypt = 'true' #for HANA Cloud

In [ ]:
import hana_ml.dataframe as dataframe

# Establish connection
conn = dataframe.ConnectionContext(address = hana_address,
                                   port = hana_port, 
                                   user = hana_user, 
                                   password = hana_password, 
                                   encrypt = hana_encrypt,
                                   sslValidateCertificate = 'false')

Through a HANA Key we are able to hide our login credentials.

In [ ]:
import hana_ml.dataframe as dataframe

# Establish connection
conn = dataframe.ConnectionContext(userkey = 'MYHANACLOUD',
                                   encrypt = 'true',
                                   sslValidateCertificate = 'false')

In [ ]:
import pandas as pd
#load data, change path to your directory
df = pd.read_csv(r"C:\<yourpath>")
df.head()

In [ ]:
#change columns to upper string
df.columns = map(str.upper, df.columns)

In [ ]:
#insert a product ID, which will later be used as key
df.insert(0, 'PRODUCT_ID', df.reset_index().index)

In [ ]:
#control a sample of the data
df.head()

In [ ]:
#Create a SAP HANA dataframe and point it to the table with the uploaded data.
df_remote = dataframe.create_dataframe_from_pandas(connection_context = conn, 
                                                   pandas_df = df, 
                                                   table_name = 'PREDICTIVEQUALITY',
                                                   force = True,
                                                   replace = False)

### 2.	How can we explore our data and react to data quality issues early? 

In [ ]:
#control the size of the data
df_remote.count()

In [ ]:
#control the variable types in SAP HANA
df_remote.dtypes()

The variable QUALITY is binary and labels all products of bad quality with a 1. Since this is a categorical variable we transform it to type NVARCHAR with the following command. 

In [ ]:
#transform the variable QUALITY
df_remote = df_remote.cast('QUALITY', 'NVARCHAR(20)')

In [ ]:
#control the variable types
df_remote.dtypes()

In [ ]:
#describe the data in SAP HANA
df_remote.describe().collect()

In [ ]:
#create a Data Report for further exploration
from hana_ml.visualizers.dataset_report import DatasetReportBuilder
datasetReportBuilder = DatasetReportBuilder()
datasetReportBuilder.build(df_remote, key="PRODUCT_ID")

In [ ]:
#generate Data Report directly in Jupyter Notebook
datasetReportBuilder.generate_notebook_iframe_report()

### 3.	How can we leverage the computing power of our HANA in our machine learning use case?  

In [ ]:
#create training and testing set
from hana_ml.algorithms.pal import partition
df_remote_train, df_remote_test, df_remote_ignore = partition.train_test_val_split(
                                                                                   random_seed = 1017,
                                                                                   data = df_remote, 
                                                                                   training_percentage = 0.8, 
                                                                                   testing_percentage = 0.2,
                                                                                   validation_percentage = 0)

In [ ]:
#control the size of the training and testing set
print('Size of training subset: ' + str(df_remote_train.count()))
print('Size of test subset: ' + str(df_remote_test.count()))

Let us now train or random forest on the training set. First, we set the numbers of trees very high, to see where the Out of Bag error converges. After optimizing the numbers of trees we will take a closer look at the variables considered at each split. 

In [ ]:
#set parameters of random forest
from hana_ml.algorithms.pal.trees import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1501, 
                            calculate_oob=True, 
                            random_state=101,
                            categorical_variable= ['SUPPLIER', 'MACHINE', 'QUALITY'])

In [ ]:
#train random forest on the training set
rf.fit(df_remote_train,
       key='PRODUCT_ID', 
       features=['SUPPLIER','MACHINE','SENSOR1','SENSOR2','SENSOR3','SENSOR4','SENSOR5','SENSOR6','SENSOR7','SENSOR8','SENSOR9','SENSOR10'], 
       label='QUALITY')

In [ ]:
#apply the trained model on the testing set
result = rf.predict(df_remote_test,
                    key='PRODUCT_ID', 
                    features=['SUPPLIER','MACHINE','SENSOR1','SENSOR2','SENSOR3','SENSOR4','SENSOR5','SENSOR6','SENSOR7','SENSOR8','SENSOR9','SENSOR10'])
result.collect()

In [ ]:
#compute confuction matrix
rf.confusion_matrix_.collect()

In [ ]:
#collect variable importance
rf.feature_importances_.collect()

In [ ]:
#collect OOB error
oob = rf.oob_error_.collect()
oob

In [ ]:
#plot the OOB error
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(oob[['TREE_INDEX']], oob[['ERROR']])

We see that the random forest converges after around 800 trees. Hence, let us now optimize the number of variables considered at each split. 

In [ ]:
#compute oob error at 801 trees for different numbers of variables considered at each split.
l = []
for var in range(0, 10):
    rf = RandomForestClassifier(n_estimators=801, calculate_oob=True, max_features=var+1, random_state=101, categorical_variable= ['SUPPLIER', 'MACHINE', 'QUALITY'])
    rf.fit(df_remote_train,key='PRODUCT_ID', features=['SUPPLIER','MACHINE','SENSOR1','SENSOR2','SENSOR3','SENSOR4','SENSOR5','SENSOR6','SENSOR7','SENSOR8','SENSOR9','SENSOR10'], label='QUALITY')
    oob = pd.DataFrame(rf.oob_error_.collect()[['ERROR']])
    l.append(oob[['ERROR']].iloc[-1])

In [ ]:
#show results
l

In [ ]:
#plot the OOB error for each number of variables considered at each split. 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.plot(l)

From our results we see that the optimal number of trees is 801 and the optimal numbers of variables considered at each split is 2. After 2 variables the OOB error increases again and we might overfitt the data. 

In [ ]:
#train the optimal random forest model
from hana_ml.algorithms.pal.trees import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=801, max_features=2,calculate_oob=True, random_state=101,categorical_variable= ['SUPPLIER', 'MACHINE', 'QUALITY'])
rf.fit(df_remote_train,key='PRODUCT_ID', features=['SUPPLIER','MACHINE','SENSOR1','SENSOR2','SENSOR3','SENSOR4','SENSOR5','SENSOR6','SENSOR7','SENSOR8','SENSOR9','SENSOR10'], label='QUALITY')
result = rf.predict(df_remote_test,key='PRODUCT_ID', features=['SUPPLIER','MACHINE','SENSOR1','SENSOR2','SENSOR3','SENSOR4','SENSOR5','SENSOR6','SENSOR7','SENSOR8','SENSOR9','SENSOR10'])
result.collect()

In [ ]:
#control confusion matrix
rf.confusion_matrix_.collect()

### 4.	How can we save and create different versions of our results

In [ ]:
#create Model storage
from hana_ml.model_storage import ModelStorage 
MODEL_SCHEMA = 'YANNICK' # HANA schema in which models are to be saved 
model_storage = ModelStorage(connection_context=conn, schema = MODEL_SCHEMA) 
rf.name = 'Random Forest Model' 
model_storage.save_model(model=rf, if_exists = 'replace')

In [ ]:
# Lists model
list_models = model_storage.list_models()
print(list_models)